In [1]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns

In [2]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [3]:
water_df.analysis_name.value_counts().to_csv("output/analysis_value_counts.csv")

In [4]:
water_df.analysis_name = [ str(i).strip() for i in water_df.analysis_name]

In [5]:
# water_df = water_df.loc[water_df['analysis_name'] == 'Base Titanium Water Analysis']
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,230,230,CO006WA0079,2012-07-19 00:00:00.000,Irrigation,sar,1.570,NaN
1,312,312,CP052WA0005,2012-07-13 00:00:00.000,Irrigation,sar,1.090,NaN
2,320,320,CP052WA0006,2012-07-13 00:00:00.000,Irrigation,sar,0.050,NaN
3,335,335,CP052WA0007,2012-07-13 00:00:00.000,Irrigation,sar,1.280,NaN
4,357,357,CP052WA0008,2012-07-13 00:00:00.000,Irrigation,sar,1.730,NaN
...,...,...,...,...,...,...,...,...
1006750,446831,446831,CU014WA0017,2020-04-09 10:34:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006751,447540,447540,CU014WA0019,2020-04-09 11:01:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006752,476832,476832,CB210WA0001,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm
1006753,477818,477818,CB210WA0002,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm


In [6]:
water_df.sort_values("batch_date")

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
739321,22111,22111,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,fluorides,0.30,ppm
377993,22108,22108,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,boron,0.01,ppm
464223,22125,22125,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,manganese,0.05,ppm
729643,22117,22117,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,calcium,9.42,ppm
488615,22116,22116,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ammonium,0.01,ppm
...,...,...,...,...,...,...,...,...
932687,392769,392769,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),silicon__,33.40,ppm
935714,398670,398670,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),total_coliforms,70.00,mpn/100ml
931118,392966,392966,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),ammonium,0.21,ppm
928797,392970,392970,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),sodium,150.00,ppm


In [7]:
water_df = water_df.sort_values('sample_code')

In [8]:
new_samples_df = water_df.loc[water_df['batch_date'] > '2024-05-21'] 
water_df = water_df.loc[water_df['batch_date'] <= '2024-05-21'] 

In [9]:
new_samples_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
925517,298335,298335,CA028WA0129,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),potassium,11.400,ppm
924745,298334,298334,CA028WA0129,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),magnesium,8.360,ppm
935044,298333,298333,CA028WA0129,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),calcium,14.800,ppm
928628,298336,298336,CA028WA0129,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),sodium,40.200,ppm
917022,298329,298329,CA028WA0129,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),electrical_conductivity,0.340,mS cm -1
...,...,...,...,...,...,...,...,...
924912,392465,392465,CV002WA0269,2024-07-26 14:58:32.680,Standard Drinking Water Analysis (WHO),magnesium,28.500,ppm
917959,396499,396499,CV002WA0269,2024-07-26 14:58:32.680,Standard Drinking Water Analysis (WHO),hardness,432.000,ppm
934438,392465,392465,CV002WA0269,2024-07-26 14:58:32.680,Standard Drinking Water Analysis (WHO),magnesium,28.500,ppm
921059,396495,396495,CV002WA0269,2024-07-26 14:58:32.680,Standard Drinking Water Analysis (WHO),molybdenum,0.083,ppm


In [10]:
water_df.analysis_name.value_counts().to_csv("output/modelling_analysis_value_counts.csv")
new_samples_df.analysis_name.value_counts().to_csv("output/test_analysis_value_counts.csv")

In [21]:
water_df['analysis_name'].nunique()

39

In [22]:
new_samples_df['analysis_name'].nunique()

6

In [23]:
new_samples_df[(new_samples_df['analysis_name'].isin(water_df['analysis_name'].unique()))].analysis_name.nunique()

6

In [24]:
water_df['analysis_name'] = [i.replace(":","") for i in water_df['analysis_name']]

In [25]:
water_df['analysis_name'].value_counts()

analysis_name
Irrigation                                          374744
Feacal Coliforms (MPN) and E.Coli                   136524
Basic Drinking Water Analysis                       121173
Standard Drinking Water Analysis                    111316
Standard Drinking Water                              63749
Reverse Osmosis Water Use                            44505
Base Titanium Water Analysis                         35185
Total Coliform and Faecal E.Coli                     28720
Standard Drinking Water Analysis (WHO)               19280
Heavy Metal Analysis (Water)                         16642
Total Dissolved Solids                               11430
Taita Water Analysis (Uni of Helsinki)                8256
Water Colour Analysis                                 7449
Heavy Metals in Water                                 6250
Feacal Coliform (MPN) and E.Coli                      4776
Total Suspended Solids (Calculated)                   2871
Turbidity                                 

In [26]:
for analysis in np.unique(water_df['analysis_name']):
    print(analysis)
    df_ = water_df.loc[water_df['analysis_name']==analysis]
    df_ = pd.pivot_table(data=df_, values="result", index="sample_code", columns="chemical_name")
    print(len(df_))

    os.makedirs(f"output/boxplots/{analysis}",exist_ok=True)
    for column in df_.columns:
        plt.boxplot(df_[column])
        plt.savefig(f"output/boxplots/{analysis}/{column}.png")
        plt.clf()
        outlier_threshold = df_[column].quantile(0.99)
        # df_ = df_.loc[df_[column] <= outlier_threshold]
    df_.describe().to_csv(f"output/chemical_null_count/{analysis}.csv")
    print(len(df_))
    
    df_.to_csv(f"output/analysis/{analysis}.csv")
    

Aquasearch Full Chemical Analysis
191
191
BT Extended Water Quality Analysis
28
28
Base Titanium Water Analysis
1548
1548
Basic Drinking Water Analysis
2720
2720
Basic Drinking Water Analysis (WHO)
338
338
Chemical & Biochemical Oxygen Demand
160
160
Chemical Oxygen Demand
79
79
Colloidal Silica in Water
30
30
E coli & Total Coliforms
105
105
Feacal Coliform (MPN) and E.Coli
214
214
Feacal Coliforms (MPN) and E.Coli
3344
3344
Free Chlorine
77
77
Heavy Metal Analysis (Water)
624
624
Heavy Metals in Water
654
654
Irrigation
6632
6632
KEBS Drinking Water Microbiology
71
71
KEBS Drinking Water Standard Analysis
94
94
KS Drinking Water Standard Analysis
19
19
Oil & Grease
63
63
Post Harvest Water Analysis
121
121
Reverse Osmosis Water Use
1785
1785
Standard Drinking Water
1420
1420
Standard Drinking Water + Free Chlorine Analysis
43
43
Standard Drinking Water Analysis
2473
2473
Standard Drinking Water Analysis (WHO)
907
907
Taita Water Analysis (Uni of Helsinki)
516
516
Total Bacteria Count

<Figure size 640x480 with 0 Axes>

In [17]:
print(len( os.listdir("output/analysis")))

39


In [ ]:
for analysis_file in os.listdir("output/analysis"):
    if f"{analysis_file.replace('.csv','')}.png" in os.listdir("./output/pairplots"):
        continue
    os.makedirs("./output/pairplots",exist_ok=True)
    
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
    print(analysis_file.replace('.csv',''))
    sns.pairplot(analysis_df)
    print('Saving')
    plt.savefig(f"output/pairplots/{analysis_file.replace('.csv','')}.png")
    print('Saved')
    plt.clf()

Irrigation


In [132]:
for analysis_file in os.listdir("output/analysis"):
    print(analysis_file)
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)

    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    analysis_df = pd.DataFrame(imp_mean.fit_transform(analysis_df), columns=analysis_df.columns, index=analysis_df.index)
    pickle.dump(imp_mean, open(f"models/imputers/{analysis_file.replace('csv','pkl')}","wb"))
    # if(len(analysis_df) == 0):
    #     continue
    sc = StandardScaler()
    pca = PCA()
    analysis_scaled = sc.fit_transform(analysis_df)
    pca.fit(analysis_scaled)
    pca_explained_variance_df = pd.DataFrame(zip(analysis_df.columns,pca.explained_variance_ratio_))
    pca_explained_variance_df.to_csv(f"output/pca_explained_variance_ratio/{analysis_file}")

    n_components = len(pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1])
    if n_components < 2:
        n_components = 2
        pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0]
    else:
        pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1]
    
    pca_chems = pca_explained_variance_df[0]
    pca = PCA(n_components=n_components)
    analysis_scaled = sc.fit_transform(analysis_df)
    pca_reduced_df = pca.fit_transform(analysis_scaled)
    analysis_scaled = pd.DataFrame(analysis_scaled,index=analysis_df.index)
    pca_reduced_df = pd.DataFrame(pca_reduced_df,index=analysis_df.index, columns=pca_chems[0:n_components])
    analysis_scaled.to_csv(f"output/analysis_scaled/{analysis_file}")
    pca_reduced_df.to_csv(f"output/pca_df/{analysis_file}")
    pickle.dump(sc, open(f"models/scalers/{analysis_file.replace('csv','pkl')}","wb"))
    pickle.dump(pca, open(f"models/pca/{analysis_file.replace('csv','pkl')}","wb"))

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'

In [133]:
# pd.DataFrame([
#     {
#         'sample_code': 'CB057WA0107',
#         'ammonium': {'result':3.936512250255515, 'units':'ppm'},
#         'bicarbonate': {'result':3.936512250255515, 'units':'ppm'},
#         'boron': {'result':3.936512250255515, 'units':'ppm'}
#     }
# ])

In [134]:
# pca_reduced_df.reset_index().to_dict('records')

In [135]:
print(len(os.listdir("output/pca")))
print(len(os.listdir("output/pca_df")))

39
39


In [136]:
for analysis_file in os.listdir("output/pca_df"):
    print(analysis_file)
    pca_reduced_df = pd.read_csv(f"output/pca_df/{analysis_file}",index_col=0)
    # if len(pca_reduced_df.columns) < 2:
        # continue
    mu = np.mean(pca_reduced_df, axis=0)
    sigma = np.cov(pca_reduced_df.T)

    pca_reduced_df['mahalanobis_distance'] = [distance.mahalanobis(pca_reduced_df.iloc[i], mu, np.linalg.inv(sigma)) for i in range(len(pca_reduced_df)) ]

    pca_reduced_df.to_csv(f"output/mahalanobis_distance/{analysis_file}")

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [137]:
analysis_file

'Water Microbiology (KEBS).csv'

In [138]:
mahalanobis_threshold_dict = {}
for analysis_file in os.listdir("output/mahalanobis_distance"):
    print(analysis_file)
    analysis = analysis_file.replace(".csv","")
    mahalanobis_df = pd.read_csv(f"output/mahalanobis_distance/{analysis_file}",index_col=0)
    upper_quantile = (mahalanobis_df['mahalanobis_distance'].quantile(0.95))
    mahalanobis_threshold_dict[analysis_file.replace(".csv","")] = upper_quantile
    mahalanobis_df.loc[mahalanobis_df['mahalanobis_distance'] >= upper_quantile].to_csv(f"output/mahanobis_distance_upper_quantile/{analysis_file}")
    plt.boxplot(mahalanobis_df['mahalanobis_distance'])
    plt.savefig(f"output/mahalanobis_boxplots/{analysis}.png")
    plt.clf()
pickle.dump(mahalanobis_threshold_dict, open("mahalanobis_thresholds.dict","wb"))

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

<Figure size 640x480 with 0 Axes>

In [139]:
len(mahalanobis_threshold_dict.keys())

38

In [140]:
new_samples_df.analysis_name.unique()

array(['Standard Drinking Water Analysis (WHO)',
       'KEBS Drinking Water Microbiology',
       'Basic Drinking Water Analysis (WHO)', 'Irrigation',
       'Standard Drinking Water + Free Chlorine Analysis',
       'Reverse Osmosis Water Use'], dtype=object)

In [11]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [12]:
reverse_analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis ORDER BY analysis_name""",con=conn_lims).set_index("analysis_name").to_dict()['analysis_id']
reverse_analysis_dict

C:\Users\tsuma.thomas\AppData\Local\Temp\ipykernel_1844\3771434913.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reverse_analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis ORDER BY analysis_name""",con=conn_lims).set_index("analysis_name").to_dict()['analysis_id']


{'% Gypsum': 540,
 '% Nitrogen (RSSP 2)': 647,
 '% Organic Matter': 258,
 '% Organic Matter (RSSP 2)': 646,
 '% Soil Nitrogen': 18,
 '%Assay': 27235,
 '%N (RSSP 2)': 652,
 '%OM (RSSP 2)': 651,
 '%P,%S Analysis(Super Calcium)': 383,
 '1:2 Soil Volume Extract': 294,
 '1:2 soil volume extract': 25,
 '1:2 Soil Volume Extract (Data Only)': 27339,
 '1:2 Vol Extract for BLGG': 317,
 '20:12:12 (N,P205,K20) Analysis': 840,
 'Absorbance at 254nm': 26936,
 'Acid Detergent Fibre': 27063,
 'Acid Insoluble Ash': 27417,
 'Acid Insoluble Matter': 27108,
 'Acid titration': 26647,
 'Acid Value': 26971,
 'Advanced Biological Farming Soil Audit': 26521,
 'Advanced Soil Health Analysis': 27181,
 'Aerobic Mesophilic Count': 26649,
 'Aflatoxin': 26794,
 'Aflatoxin AFB1': 26938,
 'Aflatoxin B1': 26939,
 'Aflatoxin in feed': 767,
 'Aflatoxin M1': 27469,
 'Aflatoxin Total': 766,
 'AfSIS Standard Leaf Analysis': 633,
 'AFSIS Std Wet Chemistry Soil Analysis': 589,
 'Aggregate Stability': 26727,
 'Agrifi Soil Micr

In [13]:
len(reverse_analysis_dict.keys())

1329

In [14]:
len(new_samples_df[['sample_code','analysis_name']].value_counts())

38

In [15]:
test = []

In [16]:
for sample in new_samples_df.sample_code.unique():
    res = {}
    tmp_ = new_samples_df.loc[new_samples_df.sample_code == sample]
    res['sample_code'] = sample
    res['analysis_id'] = [ reverse_analysis_dict[i] for i in tmp_.analysis_name.unique() if i in reverse_analysis_dict.keys() ]
    for index,row in tmp_.iterrows():
        if row['result'] >= 0 :
            res[row['chemical_name']] = {'result': row['result'], 'units': str(row['unit_name'])}
        else:
            res[row['chemical_name']] = {'result': 0, 'units': row['unit_name']}
    test.append(res)    


In [17]:
test

[{'sample_code': 'CA028WA0129',
  'analysis_id': [707],
  'potassium': {'result': 11.4, 'units': 'ppm'},
  'magnesium': {'result': 8.36, 'units': 'ppm'},
  'calcium': {'result': 14.8, 'units': 'ppm'},
  'sodium': {'result': 40.2, 'units': 'ppm'},
  'electrical_conductivity': {'result': 0.34, 'units': 'mS cm -1'},
  'sulphur': {'result': 4.61, 'units': 'ppm'},
  'phosphorus': {'result': 0.1, 'units': 'ppm'},
  'ammonium': {'result': 0.01, 'units': 'ppm'},
  'silicon__': {'result': 28.7, 'units': 'ppm'},
  'nitrate_n': {'result': 6.03, 'units': 'ppm'},
  'silica': {'result': 61.4, 'units': 'ppm'}},
 {'sample_code': 'CA028WA0130',
  'analysis_id': [707],
  'hardness': {'result': 68.6, 'units': 'ppm'},
  'silica': {'result': 58.8, 'units': 'ppm'},
  'phosphorus': {'result': 0.093, 'units': 'ppm'},
  'zinc': {'result': 0.01, 'units': 'ppm'},
  'calcium': {'result': 14.4, 'units': 'ppm'},
  'sulphate': {'result': 12.1, 'units': 'ppm'},
  'bicarbonate': {'result': 132.0, 'units': 'ppm'},
  's

In [18]:
str(test).replace("'", '"')

'[{"sample_code": "CA028WA0129", "analysis_id": [707], "potassium": {"result": 11.4, "units": "ppm"}, "magnesium": {"result": 8.36, "units": "ppm"}, "calcium": {"result": 14.8, "units": "ppm"}, "sodium": {"result": 40.2, "units": "ppm"}, "electrical_conductivity": {"result": 0.34, "units": "mS cm -1"}, "sulphur": {"result": 4.61, "units": "ppm"}, "phosphorus": {"result": 0.1, "units": "ppm"}, "ammonium": {"result": 0.01, "units": "ppm"}, "silicon__": {"result": 28.7, "units": "ppm"}, "nitrate_n": {"result": 6.03, "units": "ppm"}, "silica": {"result": 61.4, "units": "ppm"}}, {"sample_code": "CA028WA0130", "analysis_id": [707], "hardness": {"result": 68.6, "units": "ppm"}, "silica": {"result": 58.8, "units": "ppm"}, "phosphorus": {"result": 0.093, "units": "ppm"}, "zinc": {"result": 0.01, "units": "ppm"}, "calcium": {"result": 14.4, "units": "ppm"}, "sulphate": {"result": 12.1, "units": "ppm"}, "bicarbonate": {"result": 132.0, "units": "ppm"}, "sulphur": {"result": 4.03, "units": "ppm"},

In [19]:
reverse_analysis_dict[ 'KEBS Drinking Water Microbiology']

26724

In [20]:
new_samples_df.analysis_name.unique()

array(['Standard Drinking Water Analysis (WHO)',
       'KEBS Drinking Water Microbiology',
       'Basic Drinking Water Analysis (WHO)', 'Irrigation',
       'Standard Drinking Water + Free Chlorine Analysis',
       'Reverse Osmosis Water Use'], dtype=object)

In [23]:
import psycopg2

conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [24]:
analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis  ORDER BY analysis_name""",con=conn_lims).set_index("analysis_id").to_dict()['analysis_name']


C:\Users\tsuma.thomas\AppData\Local\Temp\ipykernel_1844\3258811639.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis  ORDER BY analysis_name""",con=conn_lims).set_index("analysis_id").to_dict()['analysis_name']


In [25]:
analysis_dict

{540: '% Gypsum',
 647: '% Nitrogen (RSSP 2)',
 258: '% Organic Matter',
 646: '% Organic Matter (RSSP 2)',
 18: '% Soil Nitrogen',
 27235: '%Assay',
 652: '%N (RSSP 2)',
 651: '%OM (RSSP 2)',
 383: '%P,%S Analysis(Super Calcium)',
 294: '1:2 Soil Volume Extract',
 25: '1:2 soil volume extract',
 27339: '1:2 Soil Volume Extract (Data Only)',
 317: '1:2 Vol Extract for BLGG',
 840: '20:12:12 (N,P205,K20) Analysis',
 26936: 'Absorbance at 254nm',
 27063: 'Acid Detergent Fibre',
 27146: 'Acid Insoluble Ash',
 27003: 'Acid Insoluble Ash',
 27417: 'Acid Insoluble Ash',
 27108: 'Acid Insoluble Matter',
 26647: 'Acid titration',
 26971: 'Acid Value',
 26521: 'Advanced Biological Farming Soil Audit',
 27181: 'Advanced Soil Health Analysis',
 26649: 'Aerobic Mesophilic Count',
 26794: 'Aflatoxin',
 26938: 'Aflatoxin AFB1',
 26939: 'Aflatoxin B1',
 767: 'Aflatoxin in feed',
 27469: 'Aflatoxin M1',
 26753: 'Aflatoxin Total',
 766: 'Aflatoxin Total',
 633: 'AfSIS Standard Leaf Analysis',
 589: 'AF

In [26]:
pickle.dump(analysis_dict, open("analysis.dict","wb"))

In [27]:
analysis_dict = pickle.load(open("analysis.dict","rb"))

In [28]:
analysis_dict

{540: '% Gypsum',
 647: '% Nitrogen (RSSP 2)',
 258: '% Organic Matter',
 646: '% Organic Matter (RSSP 2)',
 18: '% Soil Nitrogen',
 27235: '%Assay',
 652: '%N (RSSP 2)',
 651: '%OM (RSSP 2)',
 383: '%P,%S Analysis(Super Calcium)',
 294: '1:2 Soil Volume Extract',
 25: '1:2 soil volume extract',
 27339: '1:2 Soil Volume Extract (Data Only)',
 317: '1:2 Vol Extract for BLGG',
 840: '20:12:12 (N,P205,K20) Analysis',
 26936: 'Absorbance at 254nm',
 27063: 'Acid Detergent Fibre',
 27146: 'Acid Insoluble Ash',
 27003: 'Acid Insoluble Ash',
 27417: 'Acid Insoluble Ash',
 27108: 'Acid Insoluble Matter',
 26647: 'Acid titration',
 26971: 'Acid Value',
 26521: 'Advanced Biological Farming Soil Audit',
 27181: 'Advanced Soil Health Analysis',
 26649: 'Aerobic Mesophilic Count',
 26794: 'Aflatoxin',
 26938: 'Aflatoxin AFB1',
 26939: 'Aflatoxin B1',
 767: 'Aflatoxin in feed',
 27469: 'Aflatoxin M1',
 26753: 'Aflatoxin Total',
 766: 'Aflatoxin Total',
 633: 'AfSIS Standard Leaf Analysis',
 589: 'AF

In [29]:
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))

In [30]:
pd.DataFrame({'analysis_name':mahalanobis_thresholds.keys()}).to_csv("output/analyses_in_models.csv")

In [31]:
unit_decision = pd.read_csv("output/water_unit_per_chemical_decision.csv")

In [32]:
unit_decision

,Unnamed: 0,crop,chemical_name,unit_name
0,0,Irrigation,sar,NaN
1,0,Irrigation,nitrate_n,ppm
2,0,Irrigation,potassium,ppm
3,0,Irrigation,sulphur,ppm
4,0,Irrigation,phosphorus,ppm
...,...,...,...,...
832,0,Colloidal Silica in Water,colloidal_silica,ppm
833,0,Water Microbiology (KEBS),total_viable_count_@37,mpn/ml
834,0,KS Drinking Water Standard Analysis,total_coliforms,cfu/ml
835,0,KS Drinking Water Standard Analysis,faecal_e_coli,cfu/ml


In [33]:
len(test)

37

In [39]:
req_body = test
_ = pd.DataFrame(req_body)
unit_decision = pd.read_csv("water_unit_per_chemical_decision.csv")
analysis_dict = pickle.load(open("analysis.dict","rb"))
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))

result = {}
_df = _.explode('analysis_id', ignore_index=True)
import math
for index,row in _df.iterrows():
    sample_code = row['sample_code']
    if sample_code not in result.keys():
        result[sample_code] = []
    if math.isnan(row['analysis_id']):
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Analysis not in specified", "details": f"Analysis id not provided" })   
        continue
    row['analysis_name'] = analysis_dict[row['analysis_id']]
    analysis = row['analysis_name']
    analysis_id = row['analysis_id']
    if analysis not in mahalanobis_thresholds.keys():
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Analysis not in models", "details": f"Analysis: {analysis} is not in the list of defined models" })   
        continue     
    scaler = pickle.load(open(f"scalers/{analysis}.pkl","rb"))
    pca = pickle.load(open(f"pca/{analysis}.pkl","rb"))
    imputer = pickle.load(open(f"models/imputers/{analysis}.pkl","rb"))
    pca_df = pd.read_csv(f"output/pca_df/{analysis}.csv",index_col=0)
    analysis_df = pd.read_csv(f"output/analysis/{analysis}.csv",index_col=0)

    

    try :
        tmp_df = pd.DataFrame(row).T[analysis_df.columns]
    except:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Missing parameters for analysis_id: {analysis}", "details": f"Expected parameters are {','.join(pca_df.columns)} for analysis: {analysis}" })
        continue
    failed_units_comparison = {}    
    for col in pca_df.columns:
        expected_units = unit_decision.loc[(unit_decision['crop'] == analysis) & (unit_decision['chemical_name'] == col)]
        import math
        if type(row[col]) != dict and math.isnan(row[col]):
            failed_units_comparison[col] = {}
            failed_units_comparison[col]['expected_units'] = expected_units['unit_name'].tolist()[0]
            failed_units_comparison[col]['units_provided'] = None
            print(col)
            print(failed_units_comparison)
        elif row[col]['units'] !=   expected_units['unit_name'].values[0] :
            failed_units_comparison[col] = {}
            failed_units_comparison[col]['expected_units'] = expected_units['unit_name'].tolist()[0]
            failed_units_comparison[col]['units_provided'] = row[col]['units']
        else:
            continue    
    if len(failed_units_comparison.keys()) > 0:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Wrong units provided", "details": f"Expected units are {str(failed_units_comparison)} for analysis: {analysis}" })
        continue
    for col in analysis_df.columns:
        if type(row[col]) == dict:
            row[col] = row[col]['result']
    
    out_of_bounds_chems = [ ]
    for col in tmp_df.columns:
        if col == "sample_code":
            continue
        if col == "ec_salts":
            if row[col] > 105 or row[col] < 95:
                out_of_bounds_chems.append("ec_salts out of bounds. Allowed bounds are 95 - 105")
        if col == "Charge Balance":
            if row[col] < -1:
                out_of_bounds_chems.append("Charge Balance out of bounds. Allowed lower boundary is -10. ")       
        elif col.lower() == "total suspended solids" and row['analysis_name'].lower() == "total suspended solids":
            if row[col] > 1:
                out_of_bounds_chems.append("total suspended solids out of bounds. If analysis is total suspended solids is <1 then total suspended solids check should be <1  . ")         
    if len(out_of_bounds_chems) > 0:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Out of bounds", "details": f"{'.'.join(out_of_bounds_chems)}" })
        continue    

    tmp_df = pd.DataFrame(row).T[analysis_df.columns]
    tmp_df = imputer.transform(tmp_df)
    df_scaled = scaler.transform(tmp_df)
    df_pca = pd.DataFrame(pca.transform(df_scaled))

    mu = np.mean(pca_df, axis=0)
    sigma = np.cov(pca_df.T)

    mahalanobis_distance = distance.mahalanobis(df_pca.iloc[0], mu, np.linalg.inv(sigma))


    expected_md = mahalanobis_thresholds[analysis]

    if mahalanobis_distance > expected_md:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": "Mahalanobis distance exceeds threshold", "details":f"Mahalanobis distance of {mahalanobis_distance} exceeds threshold of {expected_md} for analysis: {analysis}" })
    else:
        result[sample_code].append({"sample_code": sample_code,"status":"pass","message": "Mahalanobis distance within threshold", "details":f"Mahalanobis distance of {mahalanobis_distance} is within threshold of {expected_md} for analysis: {analysis}" })
          

C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from vers

bicarbonate
{'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}
ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}
ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}
bicarbonate
{'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}
ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}


C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintaina

ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}
bicarbonate
{'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}
total_coliforms
{'total_coliforms': {'expected_units': 'mpn/100ml', 'units_provided': None}}
ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}


C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from vers

bicarbonate
{'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}
bicarbonate
{'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}
ammonium
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}}
bicarbonate
{'ammonium': {'expected_units': 'ppm', 'units_provided': None}, 'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}}


C:\Users\tsuma.thomas\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [41]:

len(result.keys())

37

In [42]:
len(test)

37

In [43]:
result

{'CA028WA0129': [{'sample_code': 'CA028WA0129',
   'status': 'warning',
   'message': 'Wrong units provided',
   'details': "Expected units are {'bicarbonate': {'expected_units': 'ppm', 'units_provided': None}} for analysis: Standard Drinking Water Analysis (WHO)"}],
 'CA028WA0130': [{'sample_code': 'CA028WA0130',
   'status': 'pass',
   'message': 'Mahalanobis distance within threshold',
   'details': 'Mahalanobis distance of 0.00598944503470006 is within threshold of 1.9400792834135852 for analysis: Standard Drinking Water Analysis (WHO)'}],
 'CA074WA0076': [{'sample_code': 'CA074WA0076',
   'status': 'warning',
   'message': 'Wrong units provided',
   'details': "Expected units are {'ammonium': {'expected_units': 'ppm', 'units_provided': None}} for analysis: Standard Drinking Water Analysis (WHO)"}],
 'CA309WA0185': [{'sample_code': 'CA309WA0185',
   'status': 'warning',
   'message': 'Wrong units provided',
   'details': "Expected units are {'ammonium': {'expected_units': 'ppm', 'u

In [44]:
test

[{'sample_code': 'CA028WA0129',
  'analysis_id': [707],
  'potassium': {'result': 11.4, 'units': 'ppm'},
  'magnesium': {'result': 8.36, 'units': 'ppm'},
  'calcium': {'result': 14.8, 'units': 'ppm'},
  'sodium': {'result': 40.2, 'units': 'ppm'},
  'electrical_conductivity': {'result': 0.34, 'units': 'mS cm -1'},
  'sulphur': {'result': 4.61, 'units': 'ppm'},
  'phosphorus': {'result': 0.1, 'units': 'ppm'},
  'ammonium': {'result': 0.01, 'units': 'ppm'},
  'silicon__': {'result': 28.7, 'units': 'ppm'},
  'nitrate_n': {'result': 6.03, 'units': 'ppm'},
  'silica': {'result': 61.4, 'units': 'ppm'}},
 {'sample_code': 'CA028WA0130',
  'analysis_id': [707],
  'hardness': {'result': 68.6, 'units': 'ppm'},
  'silica': {'result': 58.8, 'units': 'ppm'},
  'phosphorus': {'result': 0.093, 'units': 'ppm'},
  'zinc': {'result': 0.01, 'units': 'ppm'},
  'calcium': {'result': 14.4, 'units': 'ppm'},
  'sulphate': {'result': 12.1, 'units': 'ppm'},
  'bicarbonate': {'result': 132.0, 'units': 'ppm'},
  's

In [46]:
unit_decision.loc[unit_decision['crop'] == "Standard Drinking Water Analysis (WHO)"]

,Unnamed: 0,crop,chemical_name,unit_name
404,0,Standard Drinking Water Analysis (WHO),total_coliforms,mpn/100ml
405,0,Standard Drinking Water Analysis (WHO),e_coli,mpn/100ml
406,0,Standard Drinking Water Analysis (WHO),faecal_e_coli,cfu/100 ml
407,0,Standard Drinking Water Analysis (WHO),electrical_conductivity,mS cm -1
408,0,Standard Drinking Water Analysis (WHO),hardness,ppm
409,0,Standard Drinking Water Analysis (WHO),bicarbonate,ppm
410,0,Standard Drinking Water Analysis (WHO),sulphate,ppm
411,0,Standard Drinking Water Analysis (WHO),nitrates,ppm
412,0,Standard Drinking Water Analysis (WHO),molybdenum,ppm
413,0,Standard Drinking Water Analysis (WHO),manganese,ppm


In [53]:
final_df = pd.DataFrame()
for i in result.keys():
    out_df = (pd.DataFrame(result[i]))
    final_df = pd.concat([final_df, out_df])


In [55]:
final_df.to_csv("test_results.csv")

In [225]:
pd.DataFrame([ result[i][0]  for i in result.keys()]).to_csv("test_results.csv")